# Data Concatenator For GenCast

**필요한 Data**
- surface level ERA5, precipitation 제외
- surface level ERA5, precipitation 포함
- pressure level ERA5
- (optional) TOA solar incident radiation

cmd에서 `.nc`로 변환하는 법
```
module load gcc/cdo-1.9.3
cdo -f nc copy input.grib output.nc
```

[weatherbench2](https://console.cloud.google.com/storage/browser/weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr?inv=1&invt=Abngtw&pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22)))에서 다운로드 가능할 수도?|

In [88]:
import xarray as xr
import numpy as np
import his_utils
import dask
import cdsapi
  
# set up dataset path 
is_grib = True


if is_grib:
    ds_surface = "concat_data/suf.grib"
    ds_surface_precip = "concat_data/tp.grib"
    ds_pressure_level = "concat_data/pres.grib"
    ds_surface_precip_prior = "concat_data/tp_prior.grib"
else:
    ds_surface = "testdata/surface.nc"
    ds_surface_precip = "testdata/precip.nc"
    ds_surface_precip_prior = None
    ds_pressure_level = "testdata/pressure.nc"
# ds_TOA = "testdata/2022-01-01/2022-01-01 TOA.grib"

year = ["2021"]
month = ["06"]
day = 21
time = ["00:00", "12:00"]

if ds_surface_precip_prior is not None:
    day_prior = [f"{day-1}"]
    time_prior = [
        "12:00", "13:00", "14:00", "15:00", 
        "16:00", "17:00", "18:00", "19:00",
        "20:00", "21:00", "22:00", "23:00"]

In [ ]:
client = cdsapi.Client()
dataset = "reanalysis-era5-pressure-levels"
request = {
    "product_type": ["reanalysis"],
    "data_format": "grib",
    "download_format": "unarchived"
}

client = cdsapi.Client()

P_level_13 = [
    "50", "100", "150", "200", "250", 
    "300", "400", "500", "600","700", 
    "850", "925", "1000"
    ]

dataset_single = "reanalysis-era5-single-levels"
dataset_multi = "reanalysis-era5-pressure-levels"

req_multi = {
    "product_type": ["reanalysis"],
    "variable": [
        "geopotential",
        "specific_humidity",
        "temperature",
        "u_component_of_wind",
        "v_component_of_wind",
        "vertical_velocity"
    ],
    "year": year,
    "month": month,
    "day": [f"{day}"],
    "time": time,
    "pressure_level": P_level_13,
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': '1.0/1.0'
}

req_tp = {
    "product_type": ["reanalysis"],
    "variable": ["total_precipitation"],
    "year": year,
    "month": month,
    "day": [f"{day}"],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': '1.0/1.0'
}

req_surface = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_temperature",
        "mean_sea_level_pressure",
        "sea_surface_temperature",
        "toa_incident_solar_radiation",
        "geopotential",
        "land_sea_mask"
    ],
    "year": year,
    "month": month,
    "day":[f"{day}"],
    "time": time,
    "data_format": "grib",
    "download_format": "unarchived",
    'grid': '1.0/1.0'
}

client.retrieve(
    dataset_single,
    req_tp,
    ds_surface_precip
)

client.retrieve(
    dataset_single,
    req_surface,
    ds_surface
)

client.retrieve(
    dataset_multi,
    req_multi,
    ds_pressure_level
)

if ds_surface_precip_prior is not None:
    req_tp_prior = {
        "product_type": ["reanalysis"],
        "variable": ["total_precipitation"],
        "year": year,
        "month": month,
        "day": day_prior,
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "grib",
        "download_format": "unarchived",
    'grid': '1.0/1.0'
    }

    client.retrieve(    
        dataset_single,
        req_tp_prior,
        ds_surface_precip_prior
    )


# 1. Surface Data w/o Precipitation

precipitation 제외하고 받은 6-hrly data 전처리 하는 과정

**TODO**
- dimension 이름 변경
- 불필요한 coordinate 삭제
- variable 이름 변경 -> 나중에 합치고 한 번에 진행 가능

In [ ]:
if is_grib:
    ds1 = xr.open_dataset(ds_surface, engine='cfgrib')
    ds1 = ds1.drop_vars(['number', 'step', 'surface', 'valid_time'])
    ds1 = ds1.rename({"z" : "geopotential_at_surface"})

else:
    ds1 = xr.open_dataset(ds_surface)
    ds1 = ds1.rename({'var165':"u10",
                      'var166':"v10", 
                      'var167':"t2m", 
                      'var129':"geopotential_at_surface", 
                      'var172':"lsm", 
                      'var151':"msl", 
                      'var34': "sst",
                      'var212':"tisr"})
ds1

# 2. Precipitation Data

Accumulates 6-hour precipitation into the next time step?
or into previous step?

**TODO**
- coordinate 정리
- 강수량 합치기

In [ ]:
# 2-1. the time after
def sync_tp_coords(dataset: xr.Dataset):
    dataset = dataset.stack(new_time=['time', 'step'])
    dataset = dataset.assign_coords(new_time=dataset.valid_time.values)
    dataset = dataset.rename({'new_time': 'time'})
    dataset = dataset.drop_vars(['number', 'surface'])
    return dataset

if is_grib:
    ds2 = xr.open_dataset(ds_surface_precip, engine='cfgrib')
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior, engine='cfgrib')
        prior = sync_tp_coords(prior)

    ds2 = sync_tp_coords(ds2)
    ds2 = ds2.isel(time=slice(5,-7)) #                                          <------- FIX HERE
else:
    ds2 = xr.open_dataset(ds_surface_precip)
    ds2 = ds2.rename({'var228': 'tp'})
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior)
        prior = prior.rename({'var228': 'tp'})
ds2.time

In [ ]:
# 2-2. the time before
if is_grib and ds_surface_precip_prior is not None:
    prior = prior.isel(time=slice(5,29)) #                                        <------- FIX HERE
else:
    pass
prior.time

In [ ]:
# 2-3. merge the two datasets
if ds_surface_precip_prior is not None:
    ds2 = xr.concat([prior, ds2], dim='time')
else:
    pass

ds2 = ds2.sortby('time')
ds2 = ds2.resample(time='12h', closed='right', label='right').sum()
ds2 = ds2.isel(time=slice(2, 4)) #                                           <------- FIX HERE
ds2

# 3. Pressure Level Data

37 level data를 처리하는 과정. 17 level이어도 동일한 방식

**TODO**
- coordinate 정리

In [ ]:
if is_grib:
    ds3 = xr.open_dataset(ds_pressure_level, engine='cfgrib')
    ds3 = ds3.drop_vars(['number', 'step', 'valid_time'])
    ds3 = ds3.rename({"isobaricInhPa" : "level"})
    ds3 = ds3.sortby('level', ascending=True)
else:
    ds3 = xr.open_dataset(ds_pressure_level)
    ds3 = ds3.rename({'var130':"t", 
                      'var131':"u", 
                      'var132':"v", 
                      'var129':"z", 
                      'var133':"q", 
                      'var135':"w",
                      'plev': "level"})
    ds3['level'] = ds3['level']/100
    ds3['level'].attrs['units'] = 'hPa'
    ds3['level'].attrs['long_name'] = 'pressure level'
ds3

In [95]:
level = ds3.level.values
level = level.astype(np.int32)

ds3 = ds3.assign_coords(level = ('level', level))

# (optional) 4. TOA 가공하기

구글에서 만든 거랑 내가 다운받은거랑 같다면 상관 없음.
다를 경우에는 이거 사용해야 함.

$\therefore$ $\exists$ noise
$\Rightarrow$ 결과가 미묘하게 달라지지만 유의미해보이지는 않다

In [96]:
# 4th. TOA
# ds4 = xr.open_dataset(ds_TOA, engine='cfgrib')

# ds4 = ds4.stack(new_time=['time', 'step'])
# ds4 = ds4.assign_coords(new_time=ds4.valid_time.values)
# ds4 = ds4.rename({'new_time': 'time'})
# ds4 = ds4.drop_vars(['number', 'surface', 'valid_time'])

# 4. 3개의 데이터셋을 하나로 합성하기

*ds1*, *ds2*, *ds3*, (*ds4*)를 하나로 합치고 GC에 잘 들어가도록 다듬어주기

**TODO**
- 한 장씩 합치는 게 좋을지, 여러 장 한 번에 합치는 게 좋을지 for memory efficiency
- 합쳐서 GC에 잘 들어가는 지까지 확인

In [ ]:
# merge all datasets
# ds_list = [ds1, ds2, ds3, ds4]
ds_list = [ds1, ds2, ds3]

result = xr.merge(ds_list)

for ds in ds_list:
    ds.close()

result = his_utils.transform_dataset(result,"12hr")

## 4-1. 파일 명 정하기

In [ ]:
result

In [99]:
result_path = '/geodata2/S2S/DL/GC_input/2021-06-21/Gen_ERA5_input.nc'  #                     <------- FIX HERE

# 5. Output: GC로 준비 갈 완료!

In [100]:
result.to_netcdf(result_path)

In [ ]:
import xarray as xr
from google.cloud import storage

# xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr')
# xr.open_zarr('gs://weatherbench2/datasets/era5-hourly-climatology/1990-2017_6h_1440x721.zarr')

In [ ]:
original = xr.open_dataset("/geodata2/S2S/DL/GC_input/source-era5_date-2019-03-29_res-1.0_levels-13_steps-12.nc")
original

---
---
---
---
---
---
---
---
---
---
---
---

# 번외: xarray.resample은 어떻게 작동하는가?

| 1 | 2 | 3 | 4 | 5 | 6 | 7 | value

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | hr

`closed=` 어느 쪽을 닫힌 구간으로 쓸 것인가 $\Rightarrow$ 어느 쪽을 포함하고 반대쪽을 제외할까

`label=` sample한 거를 어느 쪽에 할당할 것인가

- case 1) `xarray.resample("6h")`

    1 + ... + 6을 0hr에 할당

- case 2) `xarray.resample("6h", closed='right', label='right')`

    2 + ... + 7을 6hr에 할당

In [ ]:
import pandas as pd

# 샘플 데이터 생성 (0시부터 23시까지)
date_range = pd.date_range(start='2021-12-31T19:00:00.000000000', end='2022-01-05T06:00:00.000000000', freq='h')
data = np.arange(1, 109, 1)
ds = pd.Series(data, index=date_range)

# 6시간 간격으로 리샘플링
ds_resampled = ds.resample('6h', closed='right', label='right').sum()

print("\n리샘플링 결과:")
print(ds_resampled)

# 각 리샘플링 구간의 시작과 끝 확인
for i, value in ds_resampled.items():
    start = i
    end = i + pd.Timedelta(hours=5)
    original_data = ds[start:end]
    print(f"\n{i}의 리샘플링 구간:")
    print(f"시작: {start}, 끝: {end}")
    print("포함된 원본 데이터:")
    print(original_data)
    print(f"합계: {value}")